In [1]:
using Pkg;Pkg.activate(".")
#Pkg.add.(["OnlineStats", "VegaDatasets", "DataFrames"])
Pkg.add(path="../")

 Activating environment at `~/Documents/juliaPkgs/JSONLines/notebooks/Project.toml`


   Updating git-repo `/home/daniel/Documents/juliaPkgs/JSONLines`
   Updating registry at `~/.julia/registries/General`


   Updating git-repo `https://github.com/JuliaRegistries/General.git`


  Resolving package versions...
Updating `~/Documents/juliaPkgs/JSONLines/notebooks/Project.toml`
  [24c2b838] + JSONLines v1.2.0 `..#refactor`
Updating `~/Documents/juliaPkgs/JSONLines/notebooks/Manifest.toml`
  [0f8b85d8] + JSON3 v1.1.1
  [24c2b838] + JSONLines v1.2.0 `..#refactor`
  [856f2bd8] + StructTypes v1.1.0


In [6]:
#Pkg.rm("JSONLines")

Updating `~/Documents/juliaPkgs/JSONLines/notebooks/Project.toml`
  [24c2b838] - JSONLines v1.2.0 `..#refactor`
Updating `~/Documents/juliaPkgs/JSONLines/notebooks/Manifest.toml`
  [0f8b85d8] - JSON3 v1.1.1
  [24c2b838] - JSONLines v1.2.0 `..#refactor`
  [856f2bd8] - StructTypes v1.1.0


In [2]:
using JSONLines, OnlineStats, VegaDatasets, DataFrames

# Get some JSONLines data

In [4]:
flights = dataset("flights-200k");

In [5]:
writelines("data/flights-200k.jsonl", flights)

# Read back in

In [3]:
flightsind = LineIndex("data/flights-200k.jsonl", nworkers = 4);

In [7]:
@assert flightsind |> Tables.rowtable == flights |> Tables.rowtable

# JSONLines functions

In [8]:
Base.displaysize() = (11, 80)

In [9]:
materialize(flightsind)

231083-element Array{NamedTuple{(:distance, :time, :delay),T} where T<:Tuple,1}:
 (distance = 405, time = 0.016666666666666666, delay = 14)
 (distance = 370, time = 5.5, delay = -11)
 (distance = 389, time = 5.666666666666667, delay = 5)
 ⋮
 (distance = 443, time = 22.616666666666667, delay = 106)
 (distance = 880, time = 22.8, delay = 125)
 (distance = 303, time = 22.95, delay = 29)

In [10]:
materialize(flightsind, row -> row[:distance]; eltype = Int)

231083-element Array{Int64,1}:
 405
 370
 389
   ⋮
 443
 880
 303

In [11]:
materialize(flightsind, row -> (;zip((:distance, :time), (row[:distance], row[:time]))...); eltype = NamedTuple{(:distance, :time), Tuple{Int, Float64}})

231083-element Array{NamedTuple{(:distance, :time),Tuple{Int64,Float64}},1}:
 (distance = 405, time = 0.016666666666666666)
 (distance = 370, time = 5.5)
 (distance = 389, time = 5.666666666666667)
 ⋮
 (distance = 443, time = 22.616666666666667)
 (distance = 880, time = 22.8)
 (distance = 303, time = 22.95)

In [5]:
JSONLines.filter(row -> row[:distance] > 400, flightsind)

LoadError: MethodError: no method matching filter(::var"#9#10", ::LineIndex{JSON3.Object{SubArray{UInt8,1,Array{UInt8,1},Tuple{UnitRange{Int64}},true},Array{UInt64,1}}})
Closest candidates are:
  filter(::Function, !Matched::DataStructures.LinkedList{T}) where T at /home/daniel/.julia/packages/DataStructures/DLSxi/src/list.jl:81
  filter(::Function, !Matched::DataStructures.MutableLinkedList{T}) where T at /home/daniel/.julia/packages/DataStructures/DLSxi/src/mutable_list.jl:92
  filter(::Any, !Matched::Tuple{Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Vararg{Any,N}} where N) at tuple.jl:267
  ...

In [ ]:
s = Series(Mean(), Variance())

In [ ]:
for row in mtcars
    fit!(s, row[:mpg])
end

In [ ]:
s

In [ ]:
c = CovMatrix()

In [ ]:
for row in mtcars
    fit!(c, zip(row[:mpg], row[:cyl]))
end

In [ ]:
c

In [ ]:
Vector{Int}(undef, 0)